In [26]:
import pymongo
import pandas as pd
import numpy as np
import json

from collections import OrderedDict

# Data Exploration

The Fraudulent Transactions dataset (sourced from https://www.kaggle.com/datasets/chitwanmanchanda/fraudulent-transactions-data) contains 6,362,620 transactions with the following features:  
* __step__ - maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (30 days simulation).

* __type__ - CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER.

* __amount__ - amount of the transaction in local currency.

* __nameOrig__ - customer who started the transaction

* __oldbalanceOrg__ - initial balance before the transaction

* __newbalanceOrig__ - new balance after the transaction

* __nameDest__ - customer who is the recipient of the transaction

* __oldbalanceDest__ - initial balance recipient before the transaction. Note that there is not information for customers that start with M (Merchants).

* __newbalanceDest__ - new balance recipient after the transaction. Note that there is not information for customers that start with M (Merchants).

* __isFraud__ - This is the transactions made by the fraudulent agents inside the simulation. In this specific dataset the fraudulent behavior of the agents aims to profit by taking control or customers accounts and try to empty the funds by transferring to another account and then cashing out of the system.

* __isFlaggedFraud__ - The business model aims to control massive transfers from one account to another and flags illegal attempts. An illegal attempt in this dataset is an attempt to transfer more than 200.000 in a single transaction.

### Uploading transaction data to MongoDB

In [33]:
# Create client and load in Fraud database
client = pymongo.MongoClient()
db = client["Fraud"]
fraud_data = db["FraudData"]

# Drop all stored documents
# fraud_data.drop()

### Example document for a transaction:

In [36]:
fraud_data.find_one()

{'_id': ObjectId('62579d2e859a15c694d31f1a'),
 'step': 1.0,
 'type': 'PAYMENT',
 'amount': 9839.64,
 'nameOrig': 'C1231006815',
 'oldbalanceOrg': 170136.0,
 'newbalanceOrig': 160296.36,
 'nameDest': 'M1979787155',
 'oldbalanceDest': 0.0,
 'newbalanceDest': '0.0',
 'isFraud': 0.0,
 'isFlaggedFraud': 0.0}

### Total documents:

In [37]:
fraud_data.count_documents({})

6362620

## Data Exploration

### Helper Functions

In [38]:
def cursor_df(cursor: pymongo.command_cursor.CommandCursor) -> pd.DataFrame:
    """
    Convert pymongo results cursor to Pandas DataFrame
    
    :param cursor: Pymongo results cursor
    :results: Pandas DataFrame with results from cursor
    """
    return pd.DataFrame(list(cursor))

### Count of fraudulent transactions:

In [39]:
# Group by if it is fraud, counting the number of transactions
stage_group_fraud = {
    "$group": {
        "_id": "$isFraud", 
        "count": {"$sum": 1}
    }
}

pipeline = [
    stage_group_fraud
]

results = fraud_data.aggregate(pipeline)

cursor_df(results)

,_id,count
0,0.0,6354407
1,1.0,8213


The dataset is very imbalanced with over 99.8% non-fraud transactions.

### Count of each transaction type:

In [40]:
# Group by transaction type, counting the number of transactions
stage_group_trans = {
    "$group": {
        "_id": {
            "transaction_type": "$type"
        },
        "count": {"$sum": 1}
    }
}

stage_project = {
    "$project": {
        "transaction_type": "$_id.transaction_type",
        "count": "$count",
        "_id": 0
    }
}

pipeline = [
    stage_group_trans,
    stage_project
]

results = fraud_data.aggregate(pipeline)

cursor_df(results)

,transaction_type,count
0,CASH_OUT,2237500
1,DEBIT,41432
2,CASH_IN,1399284
3,TRANSFER,532909
4,PAYMENT,2151495


### Fraud count by transaction:

In [41]:
# Group by transaction type, fraud, counting the number of transactions
stage_group_trans = {
    "$group": {
        "_id": {
            "transaction_type": "$type",
            "is_fraud": "$isFraud"
        },
        "count": {"$sum": 1}
    }
}

stage_project = {
    "$project": {
        "transaction_type": "$_id.transaction_type",
        "is_fraud": "$_id.is_fraud",
        "count": "$count",
        "_id": 0
    }
}

stage_sort = {
    "$sort": OrderedDict([("transaction_type", 1), ("is_fraud", 1)])
}

pipeline = [
    stage_group_trans,
    stage_project,
    stage_sort
]

results = fraud_data.aggregate(pipeline)

cursor_df(results)

,transaction_type,is_fraud,count
0,CASH_IN,0.0,1399284
1,CASH_OUT,0.0,2233384
2,CASH_OUT,1.0,4116
3,DEBIT,0.0,41432
4,PAYMENT,0.0,2151495
5,TRANSFER,0.0,528812
6,TRANSFER,1.0,4097


The only transactions that are fraudulent are __CASH_OUT__ and __TRANSFER__ transactions.

### Accounts commiting fraudulent transactions

In [48]:
# Filter only for transactions that are fraud
stage_filter_fraud = {
    "$match": {
        "isFraud": {
            "$eq": 1,
        }
    }
}

stage_project = {
    "$project": {
        "origin_account": "$nameOrig",
        "_id": 0
    }
}
    
pipeline = [
    stage_filter_fraud,
    stage_project
]

results = fraud_data.aggregate(pipeline)

fraud_accounts = cursor_df(results)["origin_account"].to_list()
len(fraud_accounts)

8213

There are no accounts that commit multiple fraudulent transactions, with there being 8,213 total fraud transactions commited by 8,213 accounts.

### Non-fraud transactions commited by fraudulent agents

In [51]:
stage_filter_non_fraud = {
    "$match": {
        "$and": [{
            "$or": [{"nameOrig": {"$in": fraud_accounts}},
                    {"nameDest": {"$in": fraud_accounts}}]
        }, 
            {"isFraud": {"$eq": 0}}]
    }
}
    
pipeline = [
    stage_filter_non_fraud
]

results = fraud_data.aggregate(pipeline)

len(cursor_df(results))

28

There are only 28 non-fraudulent transactions that involve an account that commited a fraudulent transaction.